This code is for detecting time overlap due to human error in the yaml files.

In [5]:
##use this code to copy in a new file all the text part and just keep result: {conf: 1.0, end: ,start: , word: }, {conf: 1.0, end: ,start: , word: }

#make sure that your yaml file starts with "result:" as first line and the second line is "-conf: .."
##something like this:

'''
result:
  - conf: 1.0
    end: 0.48
    start: 0.27
    word: two
'''

import re
import yaml

def remove_text_sections_from_yaml(file_path, new_file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    pattern = re.compile(r'text:.*?(?=result:)', re.DOTALL)
    modified_content = re.sub(pattern, '', content)
    modified_content = modified_content.replace('result:', '')

    # Loading the modified content as a dictionary
    data_dict = yaml.safe_load(modified_content)

    # Prepare the final content with "result:" at the beginning
    final_content = {"result": data_dict}

    # Convert the final content back to YAML format
    final_yaml = yaml.dump(final_content, sort_keys=False, allow_unicode=True)

    with open(new_file_path, 'w', encoding='utf-8') as new_file:
        new_file.write(final_yaml)
# Example usage
file_path = './transcripts/PC0004_transcript_par.yaml' # Replace with the path to your original file
new_file_path = './transcipts/newfilePC0001.yaml' # Replace with the path where you want to save the new file

# Uncomment the following line to execute the function with your file paths
#
remove_text_sections_from_yaml(file_path, file_path)


In [6]:
##Now we track overlaps
import os
import yaml

def check_overlaps(data):
    overlaps = []
    
    data = data["result"]
    data = data[0]
    # print(len(data), data)
    for i in range(len(data) - 1):
        for j in range(i + 1, len(data) - 1):
            # print(data[i], data[j])
            if data[i]["end"] > data[j]["start"]:
                overlap_case = (data[i]["word"], data[i]["start"], data[i]["end"] , data[j]["word"], data[j]["start"], data[j]["end"])
                overlaps.append(overlap_case)
    return overlaps

def main(directory_path):
    overlap_results = {}
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.yaml'):
            # print(file_name)
            file_path = os.path.join(directory_path, file_name)
            with open(file_path, 'r') as file:
                data = yaml.safe_load(file)
                overlaps = check_overlaps(data)
                if overlaps:
                    overlap_results[file_name] = overlaps
    return overlap_results

# Use the function for your directory
directory_path = './transcripts/'
overlap_results = main(directory_path)
print(overlap_results)


{}
